In [1]:
import numpy as np
import matplotlib
import os
import datetime
import scipy.io as sio

import matplotlib.cm as cm
from matplotlib import colormaps
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

import cmocean
from geopy import distance

import sys 
from PIL import Image, ImageDraw
from PIL.ExifTags import TAGS

import skimage

In [2]:
def get_data(image_file):
    with Image.open(image_file) as image:
        data = get_data_im(image)
    return data

# def get_data_im(image):
#     xmp = image.getxmp()['xmpmeta']['RDF']['Description']
    
#     data = {'lat':float(xmp['GpsLatitude']), 'lon':float(xmp['GpsLongitude']), 
#             'alt':float(xmp['AbsoluteAltitude']),'Gyaw':float(xmp['GimbalYawDegree']), 'relalt':float(xmp['RelativeAltitude'])}
#     return data

def get_data_im(image):
    xmp = image.getxmp()['xmpmeta']['RDF']['Description']
    
    data = {'lat':float(xmp['LRFTargetLat']), 'lon':float(xmp['LRFTargetLon']), 
            'alt':float(xmp['AbsoluteAltitude']),'Gyaw':float(xmp['GimbalYawDegree']), 'relalt':float(xmp['RelativeAltitude'])}
    return data

In [3]:
def euclidean(p1, p2):
    return np.sqrt((p1[0]-p2[0])**2 + (p1[1]-p2[1])**2)

def distancer(point, distance, bearing):    #changes a distance from a point from meters to coordinate units
    point2 = distance.distance(meters=distance).destination(point,bearing=bearing)
    return euclidean(point1, point2)

In [4]:
# w_m = distance.distance((y_top, x_left), (y_top, x_right)).meters
# h_m = distance.distance((y_top, x_left), (y_bottom, x_left)).meters

# scale = 3*3000/w_m

def mtopix(meters, scale):
    return int(meters*scale)

def coordtopix(point, scale, ref):  #lat log = (y,x)
    x = mtopix(distance.distance(ref, (ref[0], point[1])).meters, scale)
    y = mtopix(distance.distance(ref, (point[0], ref[1])).meters, scale)
   
    return (x, y)

def centertocorner(coords, im):
    return (int(coords[0]-im.size[0]/2), int(coords[1]-im.size[1]/2))

def rescale(alt, im, scale, fov = 73.7, tri = (5,4,3)):
    d_m = 2*np.tan((fov/2)*np.pi/180)*alt
    s = d_m/(tri[0]*1000)
    
    size = (mtopix(im.size[0]*s, scale), mtopix(im.size[1]*s, scale))
    return size
    

In [5]:
def mapper(path, xpixels = 9000, buffer = 0.002, south_ref=True, reverse=False, alt_offset = 0):
    files = os.listdir(path)
    
    if reverse:
        files.reverse()
    
    x = np.zeros(len(files))
    y = np.zeros(len(files))
    a = np.zeros(len(files))
    g = np.zeros(len(files))

    for i,file in enumerate(files):
        data = get_data(path+file)
    
        x[i] = data['lon']
        y[i] = data['lat']
        a[i] = data['alt']
        g[i] = data['Gyaw']

    #0 remover
    remove = []
    for i,lon in enumerate(x):
        if lon == 0:
            remove.append(i)

    files = [file for j, file in enumerate(files) if j not in remove]
    x =  np.delete(x, remove)
    y =  np.delete(y, remove)
    a =  np.delete(a, remove)
    g =  np.delete(g, remove)
    
    x_left = np.min(x) - buffer
    x_right = np.max(x) + buffer

    y_bottom = np.min(y) - buffer/2
    y_top = np.max(y) + buffer/2
    
    w_m = distance.distance((y_top, x_left), (y_top, x_right)).meters
    h_m = distance.distance((y_top, x_left), (y_bottom, x_left)).meters

    scale = xpixels/w_m
    
    w = mtopix(w_m, scale)
    h = mtopix(h_m, scale)
    
    combined = Image.new(mode='RGB', size=(w, h))

    for file in files:
        with Image.open(path+file) as im:
            data = get_data_im(im)
            point = (data['lat'], data['lon'])
            #alt = data['alt']
            
            
            alt = data['alt'] + alt_offset
        
            
            
            #In the future could detect roll and pitch to figure out southref
            if south_ref:
                angle = -(data['Gyaw']+180)   #south ref clockwise to north ref anticlockwise
            else:
                angle = data['Gyaw']
            
            #im_s = im.resize(rescale(alt, im, scale))

            im_s = im.resize(rescale(alt, im, scale, fov=84))

            mask = Image.new('L', im_s.size, 255)
            mask = mask.rotate(angle, expand=True)
            im_r = im_s.rotate(angle, expand=True)

            target = centertocorner(coordtopix(point, scale, ref=(y_top, x_left)), im_r)

            combined.paste(im_r, target, mask=mask)
            
    return combined, (scale, (w_m, h_m), (x_left, y_top), (x_right, y_bottom))
    

In [13]:
def give_scale(path, xpixels = 9000, buffer = 0.002, reverse=False):
    files = os.listdir(path)
    
    if reverse:
        files.reverse()
    
    x = np.zeros(len(files))
    y = np.zeros(len(files))
    a = np.zeros(len(files))

    for i,file in enumerate(files):
        data = get_data(path+file)
    
        x[i] = data['lon']
        y[i] = data['lat']
        a[i] = data['alt']

    
    #print(a)
    x_left = np.min(x) - buffer
    x_right = np.max(x) + buffer

    y_bottom = np.min(y) - buffer/2
    y_top = np.max(y) + buffer/2
    
    w_m = distance.distance((y_top, x_left), (y_top, x_right)).meters
    h_m = distance.distance((y_top, x_left), (y_bottom, x_left)).meters

    scale = xpixels/w_m
    
    w = mtopix(w_m, scale)
    h = mtopix(h_m, scale)
    
    return scale, (w_m, h_m), (x_left, y_top), (x_right, y_bottom)


In [7]:
def red_finder(image, method = 4, jump = 10, s = 1.2):
    #pixarr = testim.load()

    x = int(image.size[0]/jump)
    y = int(image.size[1]/jump)

    red_grid = np.zeros((y,x))

    for i in range(x):
        for j in range(y):
            rgb = image.getpixel((jump*i,jump*j))

            if method == 0:
                red_grid[j, i] = (2*rgb[0]-rgb[2]-rgb[1])/(sum(rgb)) if rgb[0] != 0 else -0.5
            elif method == 1:
                red_grid[j, i] = 1 if rgb[0] > s*rgb[1] and rgb[0] > s*rgb[2] else 0
            elif method == 2:
                red_grid[j, i] = (2*rgb[0]-rgb[2]-rgb[1])/(sum(rgb)) if sum(rgb) <250 and rgb[0]!=0 else -0.5
            elif method == 3:
                red_grid[j, i] = (2*rgb[0]-rgb[2]-rgb[1])/(sum(rgb)+1) if rgb[0]< 100 and sum(rgb) < 250 else -0.5
            elif method == 4:
                red_grid[j, i] = (2*rgb[0]-rgb[2]-rgb[1])/(sum(rgb)) if np.abs(rgb[2]-rgb[1])< 20 and rgb[0] != 0 else -0.5

    return red_grid

    #red_arr_grid1[j, i] = 1 if rgb[0] > 1.0*rgb[1] and rgb[0] > 1.0*rgb[2] and sum(rgb) < 250 else 0
    
def contour_finder(array, block = 1, level = 0.5):
    reduce_arr = skimage.measure.block_reduce(array,block, np.mean)
    contour = skimage.measure.find_contours(reduce_arr, level = level)
    
    return (contour, reduce_arr)
    

In [8]:
def red_contourer(grid, level = -0.05, block = 10, jump = 10, contour_threshold = 15):

    red_lvl = red_finder(grid, jump=jump, method = 4, s = 1)
    contours, reduced = contour_finder(red_lvl, block = block, level=level)

    contours2 = []
    for cont in contours:
        if cont.shape[0] > contour_threshold:
            contours2.append(cont*block*jump)

    drawer = ImageDraw.Draw(grid)

    for contour in contours2:
        contour_points = []
        
        for point in contour:
            contour_points.append((point[1], point[0]))
        
        drawer.line(contour_points, fill='yellow', width=30)

    return contours2, red_lvl, reduced

In [9]:
path = r'C:/Oceanography_data/drone_grids/'


path1 = path+'grid4n4s5/'
path2 = path+'gridall/'

path3 = path+'grid2_4n-4s-5_a/'
path4 = path+'grid2_4n-4s-5_b/'

files1 = os.listdir(path1)
files2 = os.listdir(path2)

files12 = [files1[0], files1[1]]


gridpath = r'C:/Users/rjoft/OneDrive/Desktop/Oceanography_2023/Drone/combined_grids/grid_v3/'
gridfiles = os.listdir(gridpath)

In [14]:
testgr = mapper(path+'august/test/')[0]

In [12]:
gridfiles

['combined_grid1a_v3.jpg',
 'combined_grid1a_v3_reverse.jpg',
 'combined_grid1b_v3.jpg',
 'combined_grid1b_v3_reverse.jpg',
 'combined_grid2a_v3.jpg',
 'combined_grid2a_v3_reverse.jpg',
 'combined_grid2b_v3.jpg',
 'combined_grid2b_v3_reverse.jpg']

In [17]:
with Image.open(gridpath+gridfiles[0]) as grid1:
    grid1_array = np.asarray(grid1)

In [26]:
np.array([1,1,1,1])

array([1, 1, 1, 1])

In [ ]:
scio